In [32]:
%matplotlib inline


Mạng Neural
===============

(Dịch từ bản tiếng Anh: https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py)

Khi sử dụng pytorch, package ``torch.nn`` sẽ giúp bạn xây dựng một mạng neural.

Trong bài trước, ta đã tìm hiểu về package ``autograd`` - tính đạo hàm tự động, package ``nn`` sẽ phụ thuộc vào ``autograd`` để định nghĩa mô hình và tính đạo hàm.

Để định nghĩa một mô hình mạng Neural, bạn định nghĩa một ``nn.Module`` gồm các lớp của mạng Neural và thủ tục tiến ``forward(input)`` mô tả quá trình tính toán trên dữ liệu vào và trả về kết quả.

Để ví dụ, hãy xem xét mạng neural dùng để phân loại chữ số này:

![convnet](files/ch3.png)

Đây là một mạng neural đơn giản mà dữ liệu chỉ tiến về phía trước (feed-forward): dữ liệu đầu vào (hình ảnh chữ số) được đi qua từng lớp trên mạng, và cuối cùng tính ra kết quả (là xác suất để ảnh là chữ số từ 0 đến 9).

Quá trình huấn luyện một mạng neural như vậy thường bao gồm các bước sau:

- Định nghĩa mạng neural với các tham số cần học (hay còn gọi là trọng số/weights)
- Lặp qua tập hợp các dữ liệu đầu vào
    - Cho mạng tính kết quả dựa trên dữ liệu đầu vào
    - Tính sai số của kết quả với kết quả mong đợi (còn gọi là hàm mất mát / loss function)
    - Tính đạo hàm/gradient (của hàm mất mát) trên các trọng số của mạng (bước này còn gọi là lan truyền ngược / backpropgation bởi vì việc tính đạo hàm sẽ diễn ra bằng cách lần ngược từng lớp của mạng)
    - Cập nhật giá trị mới của trọng số của mạng; chẳng hạn ta có thể dùng công thức đơn giản sau: ``weight = weight - learning_rate * gradient``

Như trong bài trước về ``autograd``, ta thấy sức mạnh của ``autograd`` sẽ thể hiện ở chỗ, bạn chỉ cần định nghĩa mạng Neural với các lớp và hàm ``forward``, việc tính đạo hàm bằng lan truyền ngược (hàm ``backward``) sẽ được định nghĩa tự động bởi ``autograd``.


Định nghĩa mạng
------------------

Giờ ta hãy định nghĩa mạng trên đây bằng pytorch

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Đầu vào: 1 channel ảnh, đầu ra: 6 channels, convolution: vuông 5x5
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Lớp biến đổi tuyến tính / kết nối hoàn toàn (fully connected): y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max-pool trên cửa sổ kích thước 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # Nếu cửa sổ là hình vuông bạn cũng có thể chỉ ghi kích thước một cạnh
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # Loại trừ đi chiều đầu tiên (kích thước của loạt dữ liệu / batch)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Như đã đề cập ở trên, bạn chỉ cần định nghĩa hàm tiến ``forward``, và hàm lan truyền ngược ``backward``
(để tính đạo hàm) sẽ được ``autograd`` tự định nghĩa giúp bạn.
Trong hàm ``forward`` bạn có thể sử dụng bất kỳ phép tính nào trên ``Tensor``.

Các trọng số cần học của mô hình sẽ được trả về qua ``net.parameters()``.

In [34]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # Trọng số của lớp convolution thứ nhất conv1

10
torch.Size([6, 1, 5, 5])


Giớ ta hãy thử sinh ra một ảnh đầu vào 32x32 ngẫu nhiên.
Chú ý: mạng trên đây yêu cầu kích thước ảnh đầu vào là 32x32.

In [35]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0703,  0.0024,  0.1143, -0.1114,  0.0750, -0.0705,  0.0376,  0.0740,
          0.0815, -0.0025]], grad_fn=<ThAddmmBackward>)


Sau đó lan truyền ngược để tính đạo hàm (đầu tiên phải gọi `zero_grad` để làm trống bộ nhớ đệm - bởi vì mặc định pytorch sẽ tích luỹ giá trị mỗi khi `backward` được gọi để tính đạo hàm)


In [36]:
net.zero_grad()
out.backward(torch.randn(1, 10))
# Giá trị đạo hàm trên trọng số của conv1
print(params[0].grad)

tensor([[[[ 0.0789, -0.0802,  0.0725,  0.0578, -0.0520],
          [-0.0380, -0.1953,  0.0825,  0.0065, -0.0445],
          [ 0.0083, -0.0156, -0.0230, -0.0029, -0.0643],
          [-0.0198, -0.0251,  0.0087,  0.1955, -0.0365],
          [-0.0376,  0.0085,  0.0276, -0.0303,  0.0968]]],


        [[[-0.0163,  0.0462,  0.0742,  0.0521, -0.1239],
          [-0.0291, -0.0697,  0.0558, -0.0039, -0.0251],
          [-0.0289,  0.0104,  0.0454,  0.0855,  0.0675],
          [ 0.0322, -0.0191,  0.0179,  0.0635, -0.0607],
          [-0.0230,  0.1631, -0.0879, -0.0708,  0.0213]]],


        [[[ 0.0494,  0.0489,  0.0968, -0.1050, -0.0229],
          [ 0.0295,  0.0226, -0.0733, -0.0063,  0.1338],
          [ 0.1096, -0.0835,  0.0511, -0.0067,  0.1064],
          [ 0.0334, -0.0353, -0.0327, -0.0292, -0.0745],
          [ 0.1014,  0.0305, -0.0195,  0.0011, -0.0249]]],


        [[[ 0.0913,  0.0086, -0.0248, -0.0151, -0.0624],
          [-0.0166, -0.0240, -0.0325, -0.0981, -0.0094],
          [ 0.0619,

<div class="alert alert-info"><h4>Chú ý</h4><p>Package ``torch.nn`` chỉ hỗ trợ xử lý dữ liệu theo loạt (mini-batch), chứ không hỗ trợ một đơn vị dữ liệu đầu vào duy nhất.

    Ví dụ, ``nn.Conv2d`` sẽ cần dữ liệu vào là Tensor 4 chiều với định dạng
    ``(số ảnh) x (số channel) x (chiều cao) x (chiều rộng)`` (còn gọi là định dạng NCHW - (num samples) x (num channels) x (height) x (width)).

    Nếu bạn có một tấm ảnh duy nhất thì chỉ cần dùng ``input.unsqueeze(0)`` để thêm vào một chiều nữa cho số lượng ảnh.</p></div>

Trước khi tiếp tục, hãy điểm lại những khái niệm đã sử dụng.

**Tóm tắt:**
  -  ``torch.Tensor`` - Mảng nhiều chiều có hỗ trợ phép tính đạo hàm tự động như ``backward()``. Đối tượng này cũng chịu trách nhiệm lưu trữ giá trị đạo hàm tính được trên Tensor này.
  -  ``nn.Module`` - Thể hiện một mạng Neural, là cách thuận tiện để mô tả các trọng số cần học, với các tiện ích để di chuyển sang GPU, xuất, mở,... 
  -  ``nn.Parameter`` - Là một loại Tensor được tự động ghi nhận là trọng số cần học khi ta định nghĩa một ``Module``.
  -  ``autograd.Function`` - Môt tả hàm tiến ``forward`` và truy ngược ``backward`` trong một phép tính toán hỗ trợ lấy đạo hàm tự động. Mỗi phép toán trên ``Tensor`` sẽ tạo ra ít nhất một đỉnh ``Function`` nối với các hàm tạo ra ``Tensor`` này (trên đồ thị tính toán) và ghi lại lịch sử quá trình tính toán.

**Đến đây, ta đã thực hiện được các bước sau của quá trình huấn luyện mạng Neural:**
  -  Định nghĩa mạng Neural
  -  Tính giá trị tiên đoán với dữ liệu đầu vào và lan truyền ngược để tính đạo hàm

**Các bước còn lại:**
  -  Tính độ mất mát
  -  Cập nhất trọng số của mạng

Hàm mất mát
-------------
Hàm mất mát nhận vào cặp `(giá trị tiên đoán, giá trị mục tiêu)` và tính sai số giữa hai giá trị.

Package nn của pytorch hỗ trợ nhiều hàm mất mát khác nhau 
(chi tiết https://pytorch.org/docs/nn.html#loss-functions).
Một hàm mất mát đơn giản là ``nn.MSELoss`` tính sai số toàn phương trung bình (mean-squared error) giữa giá trị tiên đoán và giá trị đích.

Ví dụ sau cho tính sai số MSE bằng pytorch:

In [37]:
output = net(input)
target = torch.randn(10)  # Một giá trị mục tiêu được khởi tạo ngẫu nhiên
target = target.view(1, -1)  # Làm giá trị mục tiêu có cùng kích thước với giá trị tiên đoán
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(1.6799, grad_fn=<MseLossBackward>)


Đến đây nếu ta truy ngược từ hàm mất mát  ``loss``, dùng thuộc tính
``.grad_fn`` để chỉ hàm tính đạo hàm, ta sẽ thấy đồ thị tính toán giống như sau:

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

Do đó, khi ta gọi ``loss.backward()``,  toàn bộ đồ thị sẽ được tính đạo hàm của hàm mất mát trên các trọng số cần học của mạng, và các Tensors được khai báo với ``requires_grad=True`` sẽ có thuộc tính ``.grad`` được tích luỹ với giá trị đạo hàm.

Ta hãy truy ngược vài bước từ hàm `loss` để minh hoạ đồ thị tính toán trên đây:

In [38]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Lan truyền ngược
--------
Để lan truyền ngược tính đạo hàm của hàm mất mát ta chỉ cần gọi ``loss.backward()``.
Lưu ý, đầu tiên ta cần xoá các giá trị đạo hàm hiện tại, bởi vì mặc định giá trị đạo hàm sẽ được tích luỹ .


Bây giờ ta thử gọi ``loss.backward()``, và in ra đạo hàm của trọng số `bias` của lớp `conv1` trước và sau lệnh gọi.

In [39]:
net.zero_grad()     # Xoá giá trị đạo hàm hiện tại

print('conv1.bias.grad trước khi gọi backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad sau khi gọi backward')
print(net.conv1.bias.grad)

conv1.bias.grad trước khi gọi backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad sau khi gọi backward
tensor([ 0.0285, -0.0119, -0.0106,  0.0423, -0.0105,  0.0002])


Đến đây ta đã biết cách sử dụng hàm mất mát.

**Đọc thêm:**

  Package `nn` của pytorch hỗ trợ nhiều loại modules và hàm mất mát cấu thành những yếu tố cơ bản của mạng Neural sâu. Đọc thêm tài liệu đầy đủ tại đây https://pytorch.org/docs/nn.

**Điều còn lại duy nhất:**

  - Cập nhật trọng số của mạng

Cập nhật trọng số
------------------

Phép cập nhật trọng số đơn giản nhất là phép xuống dốc ngẫu nhiên (Stochastic Gradient Descent / SGD):
     ``weight = weight - learning_rate * gradient``

Ta có thể cài đặt phép cập nhật này bằng đoạn mã đơn giản sau đây:

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

Tuy nhiên, khi học về mạng Neural, ta sẽ thấy có nhiều phép cập nhật khác (hiệu quả hơn trong nhiều trường hợp)
như SGD, Nesterov-SGD, Adam, RMSProp,... Các phép cập nhật này được hỗ trợ trong package ``torch.optim``. Cách sử dụng đơn giản nhu sau:

In [40]:
import torch.optim as optim

# Tạo bộ phận tối ưu hoá
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Cập nhật trọng số trong vòng lặp huấn luyện:
optimizer.zero_grad()   # Xoá giá trị đạo hàm hiện tại
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Cập nhật trọng số

.. Lưu ý::

      Ở đây ta cũng phải xoá giá trị đạo hàm hiện tại bằng cách gọi
      ``optimizer.zero_grad()`` bởi vì mặc định giá trị đạo hàm sẽ được tích luỹ như đã giải thích ở trên.